특성 추출 vs 특성 선택 (Feature Extraction vs Feature Selection)

특성 추출은 특성을 결합하여 더유용한 특성을 만들고(PCA)
특성 선택은 가지고 있는 특성 중에 훈련에 가장 유용한 특성을 선택한다.

1.SelectFromModel 

특성 선택을 선호할때는 특성을 얻는 비용이 높고 본래 특성의 표현을 유지하려할 때 사용한다.

특성 선택전략에는
Filter Methods 와  Wrapper Methods , Embeded Methods가 있다.

a. Fulter Methods : 적용하려는 학습 알고리즘에 상관없이 전처리 과정으로 특성 선택
- 개개의 특성과 타깃 사이에 중요한 통계적 관계가 있는지 계산, 더 깊게 관련되어 있다고 
판단되는 특성 선택
- 특성을 선택한 후 적용하려는 모델에 상관없이 사용할 수 있음.

In [28]:
'''
#1. Near Zero Variance (변수값의 분산을 보고 판단)
columns =  ['ip', 'app', 'device', 'os', 'channel',  'is_attributed']
for column in columns:
    print('{} : {}'.format(column,round(train[column].var()),2))

    
    
#2. 상관관계를 기반으로 선정
# 변수간 높은 상관관계가 존재할 경우, 두 변수가 같이 커지거나 작아지는 경향이 있다는 것
# 이 경우 모델링의 성능이 떨어지거나 모델이 불안정해질 수 있음
plt.figure(figsize=(10, 8))
sns.heatmap(data = train.corr(method='pearson'), annot=True, fmt = '.3f', linewidths=.1, cmap='Reds')


#3. 카이제곱 검정(Chi-Squared Test)
# 예측 대상이 되는 분류와 변수 간에 독립성을 검정시, 관계가 독립이라면 해당 변수는 모델링에 중요하지 않다
# 역으로 독립이 아니라면 모델링에 필요한 변수이다.
# Datetime 변수는 별도로 문자나 수치형으로 치환해야지 사용 가능함.

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
X, y = train[['ip', 'app', 'device', 'os', 'channel']], train['is_attributed']
print(train.shape)
train_new = SelectKBest(chi2, k=2).fit_transform(X, y)
print(train_new.shape)

#Modeling을 기반으로 판단(eg. Random Forest Tress내 Feature Importance 속성 사용)
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(RandomForestClassifier(n_estimators=50, random_state=42), threshold="median")

from sklearn.model_selection import train_test_split
columns = [c for c in variables if c not in ["click_time","attributed_time",'is_attributed']]
X_train, X_test, y_train, y_test = train_test_split(train[columns], train['is_attributed'], test_size=0.33, random_state=10)

select.fit(X_train[columns],y_train)
X_train_li = select.transform(train[columns])
print("X_train.shape: {}".format(X_train.shape))
print("X_train_li.shape: {}".format(X_train_li.shape))


mask = select.get_support()
print(list(zip(mask,X_train.columns)))
plt.matshow(mask.reshape(1,-1),cmap='gray_r')
plt.xlabel("Feature Number")
'''

#==================================================#
# 실습
# 1. 상단모델에 그리드서치 또는 랜덤서치로 튜닝한 모델 구성
# 최적의  R2값과 피처임포턴스 구할것

# 2. 위 쓰레드 값으로 SelectFromModel을 구해서
# 최적의 피쳐 갯수를 구할 것

# 3. 위 피쳐갯수로 데이터(피처)를 수정(삭제)해서
# 그리드 서치 또는 랜덤 서치 적용하여
# 최적의R2구할 것

# 1번값가 2번값 비교

from xgboost import XGBRegressor
from sklearn.datasets  import load_boston
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import r2_score, accuracy_score

x, y = load_boston(return_X_y = True)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, train_size = 0.8, shuffle= True, random_state=64)

#MinMax
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

#모델링/튜닝
parameters = [
    {"n_estimators" : [100, 200, 300], "learning_rate" : [0.1, 0.2, 0.001, 0.01], "max_depth" : [4,5,6]},
    {"n_estimators" : [90, 180, 270], "learning_rate" : [0.1, 0.001, 0.01], "max_depth" : [4,5,6], "colsample_bytree" : [0.6, 0.9, 1]},
    {"n_estimators" : [90, 180], "learning_rate " : [0.1, 0.001, 0.5], "max_depth" : [4,5,6], "colsample_bytree" : [0.6, 0.9, 1], "colsample_bylevel" : [0.6, 0.7, 0.9]}
]

model = RandomizedSearchCV(XGBRegressor(eval_metric = "mlogloss"), parameters, n_jobs=8, )

model.fit(x_train, y_train)
scores = model.score(x_test, y_test)
print(model.best_estimator_)
print("R2 : ", scores)

print("=============================")
models = model.best_estimator_
models.fit(x_train, y_train)
score = models.score(x_test, y_test)
print("R2 : ", score)
# XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
#              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
#              importance_type='gain', interaction_constraints='',
#              learning_rate=0.300000012, learning_rate =0.5, max_delta_step=0,
#              max_depth=6, min_child_weight=1, missing=nan,
#              monotone_constraints='()', n_estimators=110, n_jobs=20,
#              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
#              scale_pos_weight=1, subsample=1, tree_method='exact',
#              validate_parameters=1, verbosity=None)
fi = models.feature_importances_
thresholds = np.sort(fi) #XGG, RF, 등 트리류는 다있음
print(thresholds)

# R2 :  0.8657055759007868
# [0.00133573 0.00528125 0.01185187 0.01187179 0.01490339 0.01586909
#  0.01717786 0.03374418 0.04787724 0.09030636 0.10687366 0.27309474
#  0.3698129 ]

#select Feature
parameters_1 = [
    {"n_estimators" : [100, 200, 300], "max_depth" : [4,5,6]},
    {"n_estimators" : [90, 180, 270], "max_depth" : [4,5,6], "colsample_bytree" : [0.6, 0.9, 1]},
    {"n_estimators" : [90, 180], "max_depth" : [4,5,6], "colsample_bytree" : [0.6, 0.9, 1], "colsample_bylevel" : [0.6, 0.7, 0.9]}
]

for thresh in thresholds: #작은것부터 쭉죽 불러옴
    selection =  SelectFromModel(models, threshold=thresh, prefit = True)  #prefit그 값 이상의것을 다 하겠다. 

    select_x_train = selection.transform(x_train) #x_train을 select형태로 바꿔주겠다.
    print(select_x_train.shape)

    selection_model = RandomizedSearchCV(XGBRegressor(eval_metric = "mlogloss"), parameters_1, n_jobs=8)  #xgbr모델 사용
    selection_model.fit(select_x_train, y_train) # 바꾼 x_train과 y_train 다시훈련

    select_x_test = selection.transform(x_test) #x_test변환
    y_predict = selection_model.predict(select_x_test) # 

    score = r2_score(y_test, y_predict)

    print("Thresh = %.3f, n=%d, R2 : %.2f%%" %(thresh, select_x_train.shape[1], 
    score*100))


# #remove feature
# #pca
# from sklearn.decomposition import PCA
# pca = PCA()
# pca.fit(select_x_train)
# cumsum = np.cumsum(pca.explained_variance_ratio_)
# print("cumsum : " , cumsum)

# #차원 축소 쵀대치 출력
# d = np.argmax(cumsum >= 0.99) + 1
# print("cumsum >= 0.99 : ", cumsum >=0.99)
# print("d : ", d)


[20:27:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { learning_rate  } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.6,
             colsample_bynode=1, colsample_bytree=1, eval_metric='mlogloss',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints='', learning_rate=0.300000012,
             learning_rate =0.001, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=90, n_jobs=20, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_par

#특징선택
실무에서는 특징 데이터의 종류가 많거나 혹은 두가지 모두인경우가 있음
특징의 종류를 줄여보자

In [ ]:
from sklearn.datasets import fetch_rcv1
rcv_train = fetch_rcv1(subset="train")
rcv_test = fetch_rcv1(subset="test")
X_train = rcv_train.data
y_train = rcv_train.target
X_test = rcv_test.data
y_test = rcv_test.target

# Ont-Hot-Encoding된 라벨을 정수형으로 복원
classes = np.arange(rcv_train.target.shape[1])
y_train = y_train.dot(classes)
y_test = y_test.dot(classes)

print(X_train.shape)

1. 분산에 의한 선택
예측모형에서 중요한 특징 데이터는 종속데이터와의 상관관계가 크고 예측에 도움되는 데이터다, 그러나 상관관계 계산에 앞서 특징데이터의 값 자체가 표본에 따라 그다지 변하지 않으면 종속 데이터 예측에서 도움되지 않음, 따라서 표본 변화에 따른 데이터 값의 변화, 즉 분산이 기준치보다 낮은 특징 데이터는 사용하지 않아야함.
종속데이터ㅘ 특징 데이터가 모두 0또는 1 두가지만 가지는데 특징 데이터 대부분의 값이 0이라면 분류에 도움되지 않는다.

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(1e-5)
X_train_sel = selector.fit_transform(X_train)
X_test_sel = selector.transform(X_test)
X_train_sel.shape

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

In [ ]:
%%time
model = BernoulliNB()
model.fit(X_train, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test))))

In [ ]:
%%time
model = BernoulliNB()
model.fit(X_train_sel, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))

#단일 변수선택
단일 변수 선택 방법은 각각의 독립변수 하나만 사용한 예측모형의 성능일 이용하여 가장 분류성능혹은 상관관계가 높은 변수만을 선택해서 데이터를 분석하는 방법임.
sklearn feature_seletion은 지표를 여러개 제공

#==============================
chi2: 카이제곱 검정 통계값

f_classif: 분산분석(ANOVA) F검정 통계값

mutual_info_classif: 상호정보량(mutual information)
#==================================

하지만 성능좋은 변수만 사용하는 전처리기인 selectKBest클래스도 존재


In [ ]:
from sklearn.feature_selection import chi2, SelectKBest

In [ ]:
%%time

selector1 = SelectKBest(chi2, k=14330)
X_train1 = selector1.fit_transform(X_train, y_train)
X_test1 = selector1.transform(X_test)

model = BernoulliNB()
model.fit(X_train1, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train1))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test1))))

#다른모델을 이용한 중요도 계산
다른모형을 사용해서 특성 선택 후 최종 분류는 다른 모형을 사용할 수 있음.

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
%%time
n_sample = 10000
idx = np.random.choice(range(len(y_train)), n_sample)
model_sel = ExtraTreesClassifier(n_estimators=50).fit(X_train[idx, :], y_train[idx])
selector = SelectFromModel(model_sel, prefit=True, max_features=14330)
X_train_sel = selector.transform(X_train)
X_test_sel = selector.transform(X_test)

In [ ]:
%%time
model = BernoulliNB()
model.fit(X_train_sel, y_train)
print("train accuracy:{:5.3f}".format(accuracy_score(y_train, model.predict(X_train_sel))))
print("test accuracy :{:5.3f}".format(accuracy_score(y_test, model.predict(X_test_sel))))